In [7]:
import mysql.connector
import csv

# --- MySQL Connection Setup ---
db = mysql.connector.connect(
    host="localhost",          # e.g., "localhost" or "127.0.0.1"
    #user="your_username",      # your MySQL username
    #password="your_password",  # your MySQL password
    database="divvy_trips_database"   # your database name
)

cursor = db.cursor()

In [ ]:
# --- Your SQL Query ---
query = """SELECT member_casual,COUNT(*) AS total_rides,
        ROUND(MAX(ride_length_mins), 2) AS max_ride_length_mins,
        ROUND(MIN(ride_length_mins), 2) AS min_ride_length_mins,
        ROUND(AVG(ride_length_mins), 2) AS avg_ride_length_mins
        FROM analysis_table
        GROUP BY member_casual;"""  # Replace with your actual query
cursor.execute(query)

# --- Fetch Data ---
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

# --- Save to CSV ---
csv_file_name = "Preliminary summary description of rides by ride category.csv"  # Name of the CSV file to save locally

with open(csv_file_name, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(columns)      # Write column headers
    writer.writerows(rows)        # Write row data

print(f"✅ Data saved locally to '{csv_file_name}'")

✅ Data saved locally to 'Preliminary summary description of rides by ride category.csv'


In [10]:
# --- Your SQL Query ---
query = """
WITH ranked_data AS (
    SELECT 
        member_casual,
        ride_length_mins,
        ROW_NUMBER() OVER (PARTITION BY member_casual ORDER BY ride_length_mins) AS rn,
        COUNT(*) OVER (PARTITION BY member_casual) AS total_rows
    FROM analysis_table
),

quartiles AS (
    SELECT 
        member_casual,
        MIN(ride_length_mins) AS min_val,
        MAX(ride_length_mins) AS max_val,
        
        -- Approximate Q1: 25% rank
        MAX(CASE WHEN rn <= total_rows * 0.25 THEN ride_length_mins END) AS q1,
        
        -- Approximate Median: 50% rank
        MAX(CASE WHEN rn <= total_rows * 0.50 THEN ride_length_mins END) AS median,
        
        -- Approximate Q3: 75% rank
        MAX(CASE WHEN rn <= total_rows * 0.75 THEN ride_length_mins END) AS q3

    FROM ranked_data
    GROUP BY member_casual
)

SELECT 
    member_casual,
    ROUND(min_val, 2) AS min,
    ROUND(q1, 2) AS q1,
    ROUND(median, 2) AS median,
    ROUND(q3, 2) AS q3,
    ROUND(max_val, 2) AS max,

    -- Interquartile Range
    ROUND(q3 - q1, 2) AS iqr,

    -- Outlier thresholds
    CASE 
        WHEN q3 + 1.5 * (q3 - q1) > max_val THEN NULL 
        ELSE ROUND(q3 + 1.5 * (q3 - q1), 2)
    END AS upper_outlier,

    CASE 
        WHEN q1 - 1.5 * (q3 - q1) < min_val THEN NULL 
        ELSE ROUND(q1 - 1.5 * (q3 - q1), 2)
    END AS lower_outlier,

    CASE 
        WHEN q3 + 3 * (q3 - q1) > max_val THEN NULL 
        ELSE ROUND(q3 + 3 * (q3 - q1), 2)
    END AS extreme_upper_outlier,

    CASE 
        WHEN q1 - 3 * (q3 - q1) < min_val THEN NULL 
        ELSE ROUND(q1 - 3 * (q3 - q1), 2)
    END AS extreme_lower_outlier

FROM quartiles;
"""  # Replace with your actual query
cursor.execute(query)

# --- Fetch Data ---
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

# --- Save to CSV ---
csv_file_name = "Frequency distribution summary of Ride lengths.csv"  # Name of the CSV file to save locally

with open(csv_file_name, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(columns)      # Write column headers
    writer.writerows(rows)        # Write row data

print(f"✅ Data saved locally to '{csv_file_name}'")

✅ Data saved locally to 'Frequency distribution summary of Ride lengths.csv'


In [16]:
# --- Your SQL Query ---
query = """
SELECT
    CONCAT(bin_start, '-', bin_start + 1) AS ride_length_bin,
    COUNT(*) AS bin_count
FROM (
    SELECT 
        FLOOR(ride_length_mins / 2) * 2 AS bin_start
    FROM analysis_table
    WHERE member_casual = 'member'
) AS binned
GROUP BY bin_start
ORDER BY bin_start;
"""  # Replace with your actual query
cursor.execute(query)

# --- Fetch Data ---
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

# --- Save to CSV ---
csv_file_name = "Distribution data member.csv"  # Name of the CSV file to save locally

with open(csv_file_name, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(columns)      # Write column headers
    writer.writerows(rows)        # Write row data

print(f"✅ Data saved locally to '{csv_file_name}'")

✅ Data saved locally to 'Distribution data member.csv'


In [17]:
# --- Your SQL Query ---
query = """
SELECT
    CONCAT(bin_start, '-', bin_start + 1) AS ride_length_bin,
    COUNT(*) AS bin_count
FROM (
    SELECT 
        FLOOR(ride_length_mins / 2) * 2 AS bin_start
    FROM analysis_table
    WHERE member_casual = 'customer'
) AS binned
GROUP BY bin_start
ORDER BY bin_start;
"""  # Replace with your actual query
cursor.execute(query)

# --- Fetch Data ---
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

# --- Save to CSV ---
csv_file_name = "Distribution data customer.csv"  # Name of the CSV file to save locally

with open(csv_file_name, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(columns)      # Write column headers
    writer.writerows(rows)        # Write row data

print(f"✅ Data saved locally to '{csv_file_name}'")

✅ Data saved locally to 'Distribution data customer.csv'


In [ ]:
# --- Clean Up ---
cursor.close()
db.close()
print(f"✅ Database connection closed properly.")